In [2]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras import backend as K
import gc, os, datetime
import plotly.express as px

2025-11-06 00:54:17.637680: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-06 00:54:17.690954: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-06 00:54:20.920814: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
spark = SparkSession.builder \
        .appName("LSTM_Index_Incremental") \
        .config("spark.jars.packages", "org.postgresql:postgresql:42.2.23") \
        .config("spark.driver.memory", "6g") \
        .getOrCreate()

:: loading settings :: url = jar:file:/media/bazzi/Bazzi/SSI_PROJECT/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/bazzi/.ivy2/cache
The jars for the packages stored in: /home/bazzi/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6963d7e2-3db7-4d05-8298-1d76326c2690;1.0
	confs: [default]
	found org.postgresql#postgresql;42.2.23 in central
	found org.checkerframework#checker-qual;3.5.0 in central
:: resolution report :: resolve 152ms :: artifacts dl 6ms
	:: modules in use:
	org.checkerframework#checker-qual;3.5.0 from central in [default]
	org.postgresql#postgresql;42.2.23 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

In [4]:
raw_db_url = "jdbc:postgresql://localhost:5432/ssi_db"
raw_db_properties = {
        "user": "bazzi",
        "password": "bazzi123",
        "driver": "org.postgresql.Driver"
    }

dw_db_url = "jdbc:postgresql://localhost:5432/data_warehouse_ssi"
dw_db_properties = {
        "user": "bazzi",
        "password": "bazzi123",
        "driver": "org.postgresql.Driver"
    }

date_dw_df = spark.read.jdbc(url=dw_db_url, table="dim.date", properties=dw_db_properties)
time_dw_df = spark.read.jdbc(url=dw_db_url, table="dim.time", properties=dw_db_properties)
index_dw_df = spark.read.jdbc(url=dw_db_url, table="dim.index", properties=dw_db_properties)
exchange_dw_df = spark.read.jdbc(url=dw_db_url, table="dim.exchange", properties=dw_db_properties)
session_dw_df = spark.read.jdbc(url=dw_db_url, table="dim.tradingsession", properties=dw_db_properties)


In [5]:
index_raw_df = spark.read.jdbc(url=raw_db_url, table="streaming.index_data", properties=raw_db_properties)

fact_index = index_raw_df.alias("ir") \
    .join(date_dw_df.alias("d"), col("ir.trading_date") == col("d.tradingdate"), "left") \
    .join(time_dw_df.alias("t"), col("ir.time") == col("t.time_hh_mm_ss"), "left") \
    .join(index_dw_df.alias("i"), col("ir.index_name") == col("i.index_name"), "left") \
    .join(exchange_dw_df.alias("e"), col("ir.exchange") == col("e.exchange_name"), "left") \
    .join(session_dw_df.alias("s"), col("ir.trading_session") == col("s.trading_session"), "left")

In [6]:
fact_indexmarket_df = fact_index.select(
        col("d.tradingdate"),
        col("t.time_hh_mm_ss"),
        col("i.index_name"),
        col("ir.index_value").alias("index_value"),
        col("ir.prior_index_value").alias("prio_index_value"),
        col("ir.change").alias("change"),
        col("ir.ratio_change").alias("ratio_change"),
        col("ir.total_qtty").alias("total_qtty"),
        col("ir.total_value").alias("total_value"),
        col("ir.total_qtty_pt").alias("total_qtty_pt"),
        col("ir.total_value_pt").alias("total_value_pt"),
        col("ir.advances").alias("advances"),
        col("ir.nochanges").alias("nochanges"),
        col("ir.declines").alias("declines"),
        col("ir.ceilings").alias("ceilings"),
        col("ir.floors").alias("floors"),
        col("e.exchange_key").alias("exchange_key"),
        col("s.trading_session_key").alias("trading_session_key")
    ).distinct()

In [7]:
pandas_df = fact_indexmarket_df.toPandas()

In [8]:
print(pandas_df.head())

  tradingdate       time_hh_mm_ss index_name index_value prio_index_value  \
0  2025-11-04 1970-01-01 09:18:25     VNREAL   1999.7700        1981.2300   
1  2025-11-04 1970-01-01 09:20:10     VNHEAL   2104.8000        2113.9600   
2  2025-11-04 1970-01-01 09:22:30  VNDIAMOND   2596.6100        2612.8200   
3  2025-11-04 1970-01-01 09:24:35     VNHEAL   2099.1500        2113.9600   
4  2025-11-04 1970-01-01 09:25:00      VNENE    556.1700         557.7600   

   change  ratio_change  total_qtty      total_value total_qtty_pt  \
0   18.54          0.94  4583251.00   96214486170.00          0.00   
1   -9.16         -0.43    87250.00    3028277000.00          0.00   
2  -16.21         -0.62  7128423.00  287040488950.00          0.00   
3  -14.81         -0.70   133550.00    4098060000.00          0.00   
4   -1.59         -0.29   791667.00   18290018430.00          0.00   

  total_value_pt advances nochanges declines ceilings  floors  exchange_key  \
0           0.00   0.0000    0.0000  

In [9]:
index_name = "VNINDEX"
df_index = pandas_df[pandas_df["index_name"] == index_name].sort_values('tradingdate')

fig = px.line(df_index, x='tradingdate', y='index_value', title=f'Chỉ số {index_name}')
fig.show()